# __XGBoost 이용하기__
---
- 최초 작성일: 20-10-29
- 수정: 20-12-21

In [2]:
# 결과 기록할 파일 만들기
df = pd.DataFrame(columns = ['식당', '모델','메뉴 포함 여부', 'gate포함 여부', '조중석야 구분', '배식구 구분',
                             '학습기간 설명', '학습기간', '테스트 기간', '테스트 횟수',
                             '예측값_월', '예측값_화', '예측값_수', '예측값_목', '예측값_금', 'RMSE', '사용한 변수'] )
df.to_csv('xgb_summary_df.csv', index= False, encoding = 'utf-8-sig')

NameError: name 'pd' is not defined

In [21]:
class ldccxgb():
      
    def __init__(self, **kwargs):
        import pandas as pd
        import numpy as np
        from sklearn.metrics import mean_squared_error
        import xgboost as xgb
        from xgboost import XGBClassifier
        from xgboost import plot_importance
        from sklearn.model_selection import train_test_split

        
        ### warning 메시지 무시
#         import warnings
#         warnings.filterwarnings(action='ignore')
        
#         plt.rc('font', family='NanumGothic')
#         print(plt.rcParams['font.family'])

#         graph_params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (20, 8),
#          'axes.labelsize': 'x-large',
#          'axes.titlesize':'x-large',
#          'xtick.labelsize':'x-large',
#          'ytick.labelsize':'x-large',
#          'axes.grid' : True}
#         plt.rcParams.update(graph_params)

    def get_parameter(self, common_dict, para_dict, para2_dict, info_dict):
        self.train_start_date = common_dict['train_start_date']
        self.train_end_date = common_dict['train_end_date']
        self.test_start_date = common_dict['test_start_date']
        self.test_end_date = common_dict['test_end_date']
        self.explain = common_dict['explain']
        self.model_name = common_dict['model_name']
        
        self.params = para_dict
        self.num_rounds = para2_dict['num_rounds']
        
        self.cafeteria = info_dict['식당']
        self.bldn = info_dict['조중석야']
        self.counter = info_dict['배식구 구분']
        
    ### local에서 데이터 가져오기    
    def data_import(self, data_location, columns_list):
        
        #self.merged_df = pd.read_csv('lottefood_input2.csv')
        self.columns_list = columns_list
        self.df = pd.read_csv(data_location)
        self.df = self.df.astype({'timestamp':'datetime64[ns]'})

       
        # target_value 컬럼 맨 뒤로 빼기
        self.columns_list.remove('target_value')
        self.columns_list.append('target_value')
        
        self.df = self.df[self.columns_list]

    def data_prep(self):    
        
        ## local에서 train 데이터 만들기
        
        self.test_df = self.df[
            (self.df['timestamp'] >= self.test_start_date)
            &
            (self.df['timestamp'] <= self.test_end_date)
        ]
           
        ## train 데이터 학습 날짜 선정
        self.train_df = self.df[
            (self.df['timestamp'] >= self.train_start_date) 
            & 
            (self.df['timestamp'] <= self.train_end_date)
        ]
               
        
        # timestamp drop하기
        
        self.train_df = self.train_df.drop('timestamp', axis = 1)
        self.test_df = self.test_df.drop('timestamp', axis = 1)
        
        self.train_x = self.train_df.drop('target_value', axis = 1)
        self.train_y = self.train_df[['target_value']]
        
        self.test_x = self.test_df.drop('target_value', axis = 1 )
        self.test_y = self.test_df[['target_value']]

        self.train_x, self.eval_x, self.train_y, self.eval_y = train_test_split(self.train_x, self.train_y, test_size = 0.2)
        print(self.train_x.shape, self.eval_x.shape)

        self.dtrain = xgb.DMatrix(data = self.train_x, label = self.train_y)
        self.deval = xgb.DMatrix(data = self.eval_x, label = self.eval_y)
        
        self.dtest = xgb.DMatrix(data = self.test_x)
        

    def train_model(self):
        # train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
        wlist = [ (self.dtrain,'train'), (self.deval,'eval') ]
        
        # 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
        self.xgb_model = xgb.train(params = self.params , dtrain= self.dtrain , num_boost_round = self.num_rounds, evals= wlist )
        
        xgb.plot_tree(self.xgb_model, num_trees=0)
        plt.show()


    def run_pred(self):
        self.pred = self.xgb_model.predict(self.dtest)

        
    def visualize_rst(self):
        
        self.y_pred = self.pred.tolist()
        self.y_true = self.test_y['target_value'].tolist()
        timestamp_df = pd.date_range(start= self.test_start_date, end= self.test_end_date)
        timestamp_df = pd.DataFrame({'timestamp': timestamp_df})
        self.result_df = pd.DataFrame({"y_true":self.y_true, "y_pred": self.y_pred})
        self.result_df = pd.concat([timestamp_df, self.result_df], axis = 1)
        
        print(self.result_df)
        
        # 시각화
        plt.title('예측 결과', fontsize = 20)
        plt.plot(self.result_df['timestamp'], self.result_df['y_true'], label = '실제')
        plt.plot(self.result_df['timestamp'], self.result_df['y_pred'], label= '예측', linewidth = 2, linestyle = '--')
        # plt.bar(result_df['SALE_DATE'], result_df['difference'], alpha = 0.3, label = '차이 (실제 - 예측)')
        plt.xticks(rotation=45)
        plt.legend()
        
        # feature_importance 그래프
        
        plot_importance(self.xgb_model)
    
        
    def error_metric(self):
        self.y_pred = self.result_df['y_pred'].tolist()
        self.y_true = self.result_df['y_true'].tolist()
        self.rmse = mean_squared_error(self.y_pred, self.y_true)**0.5
        
    
    def save_summary(self):
        self.summary_df = pd.read_csv('xgb_summary_df.csv')

              
        info_lst = []
        info_lst.append(self.cafeteria)
        info_lst.append(self.model_name)
        menu_included = '미포함'
        gate_included = '미포함'
        
        for column in self.columns_list:
            if 'menu' in column:
                menu_included = '포함'
                break;

        for column in self.columns_list:
            if 'gate' in column:
                gate_included = '포함'
                break;
        
        info_lst.append(menu_included)
        info_lst.append(gate_included)
        info_lst.append(self.bldn)
        info_lst.append(self.counter)
        info_lst.append(self.explain)
        
        # 학습 기간 추가
        train_period = str(self.train_start_date)+'~'+str(self.train_end_date)
        info_lst.append(train_period)
        
        # 테스트 기간 추가
        test_period = self.test_start_date+'~'+self.test_end_date
        info_lst.append(test_period)
        
        # 횟수
        info_lst.append(' ')
        # 예측값
        for i in range(5):
            info_lst.append(self.result_df.loc[i, 'y_pred'])
        # RMSE
        info_lst.append(self.rmse)
        
        # 사용한 변수
        info_lst.append(self.columns_list)

        self.summary_df = self.summary_df.append(pd.Series(info_lst, index = self.summary_df.columns), ignore_index = True)
        
        self.summary_df.to_csv('xgb_summary_df.csv', index = False, encoding = 'utf-8-sig')
                                     

    def show_param(self):
        print(' - train start: ', self.train_start_date)
        print(' - train end: ', self.train_end_date)
        print(' - test start: ', self.test_start_date)
        print(' - test end: ', self.test_end_date)        
        print(' - 모델명: ', self.model_name)
        print('\n')
        print(' - 실제 값: ', self.y_true)
        print(' - 예측 값: ', self.y_pred)
        print('================================================')
        print('rmse : ', self.rmse)
        print('================================================')
        print('\n')
        print('\n')
        print('\n')
        print('\n')
        print('\n')

            

######################################################################################################



In [22]:
def predict():
    if __name__ == '__main__':

        ### 0. xgboost 객체 생성
        ldccxgb = ldccXGB()
        
        ### 1. 파라미터 입력하기
        
          # 9월 1주차
#         common_dict = {'explain': '9월 1주차',
                       
#                        'train_start_date': '2019-01-01', 'train_end_date': '2020-08-27',
                       
#                        'test_start_date': '2020-08-31' , 'test_end_date': '2020-09-04',
                       
#                        'model_name': 'xgboost'}
        
        # 강수량 주간
        common_dict = {'explain': '강수량 주간',

                       'train_start_date': '2019-01-01', 'train_end_date': '2020-07-30',
                      
                       'test_start_date': '2020-08-03' , 'test_end_date': '2020-08-07',
                       
                       'model_name': 'xgboost'}        
        
#        # 연휴 주간
#         common_dict = {
#                        'explain' : '연휴 주간',

#                        'train_start_date': '2019-01-01', 'train_end_date': '2020-08-13',
                       
#                        'test_start_date': '2020-08-17' , 'test_end_date': '2020-08-21',
                       
#                        'model_name': 'xgboost'}

        # xgb 파라미터
        # max_depth : Tree 깊이 수
        # max_leaf_node: 하나의 Tree에서 노드 수
        para_dict = {
            'max_depth':4, 'learning_rate':0.1, 'early_stoppings' : 30
        }
        
        para2_dict = {'num_rounds' : 100}
        
        info_dict = {'식당' : '천안', '조중석야' : '중식', '배식구 구분': '없음' }
        
        ldccxgb.get_parameter(common_dict, para_dict, para2_dict, info_dict) 
    
        col_lst = ['timestamp', 'target_value', 'dayofweek_mean', 'dayofweek_median','temperature', 'rainfall', 'holiday', 'before_holiday', 'after_holiday',
       'gate', 'proportion', 'weekday_Fri', 'weekday_Mon', 'weekday_Sat','weekday_Sun', 'weekday_Thur', 'weekday_Tue', 'weekday_Wed']
        
        ldccxgb.data_import('천안input.csv', col_lst)

        ### 3. 데이터 전처리
        ldccxgb.data_prep()

        ### 4. 모델 학습
        ldccxgb.train_model()

        ### 5. 예측 수행
        ldccxgb.run_pred()

        ### 6. 결과 시각화
        ldccxgb.visualize_rst()

        ### 7. 에러 메트릭 생성
        ldccxgb.error_metric()

        ### 8. 결과 저장

        ldccxgb.save_summary()

        ### 9. 파라미터 출력
        ldccxgb.show_param()



    return 0

#####################반복#########################

for i in range(5):
    predict()
print('completed')

NameError: name 'train_test_split' is not defined

In [18]:
import pandas as pd
data=pd.read_csv('천안input.csv',index_col=0)
data.columns

Index(['timestamp', 'target_value', 'dayofweek_mean', 'dayofweek_median',
       'temperature', 'rainfall', 'holiday', 'before_holiday', 'after_holiday',
       'gate', 'proportion', 'weekday_Fri', 'weekday_Mon', 'weekday_Sat',
       'weekday_Sun', 'weekday_Thur', 'weekday_Tue', 'weekday_Wed'],
      dtype='object')